# Importación de librerias

In [129]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re

import sklearn
from sklearn import set_config
set_config(transform_output = "pandas")
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.impute import KNNImputer, SimpleImputer


In [2]:
print("Versión pandas",pd.__version__)
print("Version seaborn", sns.__version__)
!python --version

Versión pandas 2.0.2
Version seaborn 0.12.2
Python 3.11.3


# Cargar datos

In [3]:
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD,"data")
FILE_NAME = "2017_accidents_causa_conductor_gu_bcn_.csv"
accidents_causa = os.path.join(DATA_PATH,FILE_NAME)

In [4]:
CWD_1 = os.getcwd()
DATA_PATH_1 = os.path.join(CWD_1,"data")
FILE_NAME_1 = "2017_lloguer_preu_trim.csv"
lloguer = os.path.join(DATA_PATH_1,FILE_NAME_1)

In [5]:
CWD_2 = os.getcwd()
DATA_PATH_2 = os.path.join(CWD_2,"data")
FILE_NAME_2 = "2017_poblacio_exposada_barris_mapa_estrategic_soroll_bcn_long.csv"
poblacio_soroll = os.path.join(DATA_PATH_2,FILE_NAME_2)

In [6]:
CWD_3 = os.getcwd()
DATA_PATH_3 = os.path.join(CWD_3,"data")
FILE_NAME_3 = "2017_sup_mitjana_habitatge.csv"
superficie_habitatge = os.path.join(DATA_PATH_3,FILE_NAME_3)

In [7]:
CWD_4 = os.getcwd()
DATA_PATH_4 = os.path.join(CWD_4,"data")
FILE_NAME_4 = "h2malloa2011.csv"
precio_alquiler = os.path.join(DATA_PATH_4,FILE_NAME_4)

In [8]:
df_accidents = pd.read_csv(accidents_causa)
df_lloguer = pd.read_csv(lloguer)
df_poblacio = pd.read_csv(poblacio_soroll)
df_superficie = pd.read_csv(superficie_habitatge, encoding='latin-1')

# Funciones

In [9]:
# Creamos una función simple para hacer un report simple sobre los DataFrame suministrados
def report_df(dataframe, verbose = True):
    print("El tamaño del dataset es", dataframe.shape[0], "filas y", dataframe.shape[1], "columnas")
    print()
    print(dataframe.info(verbose = verbose))
    total_nulos = dataframe.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")


In [10]:
# Función para convertir columnas categoricas a numéricas de cara a preprocesar los datos y preparlos para un modelo
def OHE (dataframe, column_name):
    _dummy_dataset = pd.get_dummies(dataframe[column_name],prefix=column_name)
    dataframe = pd.concat([dataframe,_dummy_dataset],axis=1)
    return dataframe.drop(column_name,axis=1)

In [126]:
def juntar_etiquetas(dataframe,column,value_contain,new_value):
    dataframe.loc[dataframe[column].str.contains(value_contain), column] = new_value

# Análisis df_superficie

In [11]:
report_df(df_superficie)

df_superficie.head(2)

El tamaño del dataset es 73 filas y 6 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Any                                73 non-null     int64  
 1   Codi_districte                     73 non-null     int64  
 2   Nom_districte                      73 non-null     object 
 3   Codi_barri                         73 non-null     int64  
 4   Nom_barri                          73 non-null     object 
 5   Superficie_mitjana_habitatge_(m2)  73 non-null     float64
dtypes: float64(1), int64(3), object(2)
memory usage: 3.6+ KB
None

Tenemos un total de 0 nulos


,Any,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Superficie_mitjana_habitatge_(m2)
0,2017,1,Ciutat Vella,1,el Raval,65.6
1,2017,1,Ciutat Vella,2,el Barri Gòtic,88.2


# df_accidents -- análisis y limpieza

In [12]:
report_df(df_accidents)

df_accidents.head(2)

El tamaño del dataset es 11091 filas y 20 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11091 entries, 0 to 11090
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Numero_expedient            11091 non-null  object 
 1   Codi_districte              11091 non-null  int64  
 2   Nom_districte               11091 non-null  object 
 3   Codi_barri                  11091 non-null  int64  
 4   Nom_barri                   11091 non-null  object 
 5   Codi_carrer                 11091 non-null  int64  
 6   Nom_carrer                  11091 non-null  object 
 7   Num_postal                  11089 non-null  object 
 8   Descripcio_dia_setmana      11091 non-null  object 
 9   NK_Any                      11091 non-null  int64  
 10  Mes_any                     11091 non-null  int64  
 11  Nom_mes                     11091 non-null  object 
 12  Dia_mes                     11091 no

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,NK_Any,Mes_any,Nom_mes,Dia_mes,Hora_dia,Descripcio_torn,Descripcio_causa_conductor,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
0,2017S004939,-1,Desconegut,-1,Desconegut,-1,Motors ...,43-51,Dimarts,2017,6,Juny,6,8,Desobeir altres senyals,Matí,427585.89,4577869.16,2.191767,41.411606
1,2017S007937,-1,Desconegut,-1,Desconegut,-1,Joan XXIII / Martí i Franquès ...,NaN,Dimarts,2017,9,Setembre,26,9,Gir indegut o sense precaució,Matí,426505.49,4581655.96,2.199239,41.419635


In [13]:
df_accidents.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
Codi_districte,11091.0,4.760256e+00,3.048208,-1.000000e+00,2.000000e+00,4.000000e+00,7.000000e+00,1.000000e+01
Codi_barri,11091.0,2.660346e+01,22.200196,-1.000000e+00,8.000000e+00,1.900000e+01,4.300000e+01,7.300000e+01
Codi_carrer,11091.0,2.410163e+05,193940.152293,-1.000000e+00,1.100090e+05,1.912040e+05,3.124000e+05,7.018640e+05
NK_Any,11091.0,2.017000e+03,0.000000,2.017000e+03,2.017000e+03,2.017000e+03,2.017000e+03,2.017000e+03
Mes_any,11091.0,6.505816e+00,3.435091,1.000000e+00,4.000000e+00,6.000000e+00,1.000000e+01,1.200000e+01
Dia_mes,11091.0,1.580236e+01,8.757472,1.000000e+00,8.000000e+00,1.600000e+01,2.300000e+01,3.100000e+01
Hora_dia,11091.0,1.376035e+01,5.300569,0.000000e+00,1.000000e+01,1.400000e+01,1.800000e+01,2.300000e+01
Coordenada_UTM_X_ED50,11091.0,4.301166e+05,2030.298735,4.242081e+05,4.286143e+05,4.301559e+05,4.315727e+05,4.351199e+05
Coordenada_UTM_Y_ED50,11091.0,4.583624e+06,2346.034874,4.575198e+06,4.582190e+06,4.583336e+06,4.584891e+06,4.591264e+06
Longitud,11091.0,2.162875e+00,0.024180,2.091855e+00,2.144985e+00,2.163448e+00,2.180188e+00,2.222572e+00


In [14]:
df_accidents.describe(exclude=np.number).T

,count,unique,top,freq
Numero_expedient,11091,10324,2017S004579,4
Nom_districte,11091,11,Eixample,3203
Nom_barri,11091,74,la Dreta de l'Eixample,1139
Nom_carrer,11091,1594,Corts Catalanes,449
Num_postal,11089,2107,0001 0001,231
Descripcio_dia_setmana,11091,7,Divendres,1880
Nom_mes,11091,12,Novembre,1090
Descripcio_torn,11091,16,Manca d'atenció a la conducció,1920
Descripcio_causa_conductor,11091,3,Tarda,5389


In [15]:
df_accidents.columns

Index(['Numero_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri',
       'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal ',
       'Descripcio_dia_setmana', 'NK_Any', 'Mes_any', 'Nom_mes', 'Dia_mes',
       'Hora_dia', 'Descripcio_torn', 'Descripcio_causa_conductor',
       'Coordenada_UTM_X_ED50', 'Coordenada_UTM_Y_ED50', 'Longitud',
       'Latitud'],
      dtype='object')

#### En el DataFrame de accidents, he visto que hay columnas que tienen un espacio blanco  al final(invisible a priori), que genera problemas para buscar/filtrar si no lo tenemos en cuenta, como se puede observar en la siguiente línea de código, así que buscaré que columnas les sucede eso y las renombrare para evitar futuros problemas de búsqueda. Ya que estamos limpiamos y aplicamos los espacios para todos los Dataframe

In [16]:
'Num_postal ' # Tiene espacio en blanco al final, antes del cierre de comillas

'Num_postal '

In [17]:
df_accidents.columns = df_accidents.columns.str.replace(" ","")
df_lloguer.columns = df_lloguer.columns.str.replace(" ","")
df_poblacio.columns = df_poblacio.columns.str.replace(" ","")

In [18]:
df_accidents[df_accidents["Num_postal"].isnull()]

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,NK_Any,Mes_any,Nom_mes,Dia_mes,Hora_dia,Descripcio_torn,Descripcio_causa_conductor,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
1,2017S007937,-1,Desconegut,-1,Desconegut,-1,Joan XXIII / Martí i Franquès ...,NaN,Dimarts,2017,9,Setembre,26,9,Gir indegut o sense precaució,Matí,426505.49,4581655.96,2.199239,41.419635
3,2017S008779,-1,Desconegut,-1,Desconegut,-1,Número 6 Zona Franca / A Zona Franca ...,NaN,Dilluns,2017,10,Octubre,23,22,Desobeir altres senyals,Nit,427519.99,4575229.36,2.190955,41.406769


In [19]:
df_accidents[df_accidents['Nom_barri']=='Desconegut']

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,NK_Any,Mes_any,Nom_mes,Dia_mes,Hora_dia,Descripcio_torn,Descripcio_causa_conductor,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
0,2017S004939,-1,Desconegut,-1,Desconegut,-1,Motors ...,43-51,Dimarts,2017,6,Juny,6,8,Desobeir altres senyals,Matí,427585.89,4577869.16,2.191767,41.411606
1,2017S007937,-1,Desconegut,-1,Desconegut,-1,Joan XXIII / Martí i Franquès ...,NaN,Dimarts,2017,9,Setembre,26,9,Gir indegut o sense precaució,Matí,426505.49,4581655.96,2.199239,41.419635
2,2017S004739,-1,Desconegut,-1,Desconegut,-1,Corts Catalanes ...,900,Dilluns,2017,5,Maig,29,22,Manca d'atenció a la conducció,Nit,432587.59,4584475.05,2.186875,41.412198
3,2017S008779,-1,Desconegut,-1,Desconegut,-1,Número 6 Zona Franca / A Zona Franca ...,NaN,Dilluns,2017,10,Octubre,23,22,Desobeir altres senyals,Nit,427519.99,4575229.36,2.190955,41.406769
4,2017S004350,-1,Desconegut,-1,Desconegut,-1,Aguilar ...,7-9,Dimarts,2017,5,Maig,16,14,Altres,Tarda,430758.19,4586316.94,2.186557,41.409004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9730,2017S003352,-1,Desconegut,-1,Desconegut,-1,Desconegut,Desconegut,Dimecres,2017,4,Abril,12,20,Manca precaució incorporació circulació,Tarda,433202.22,4584544.05,2.199680,41.407824
9731,2017S006639,-1,Desconegut,-1,Desconegut,-1,Desconegut,Desconegut,Dilluns,2017,7,Juliol,31,8,Manca d'atenció a la conducció,Matí,432760.08,4583622.69,2.194493,41.399488
9732,2017S003278,-1,Desconegut,-1,Desconegut,-1,Desconegut,Desconegut,Dilluns,2017,4,Abril,10,9,Manca d'atenció a la conducció,Matí,427183.33,4581831.13,2.128001,41.382867
9733,2017S005635,-1,Desconegut,-1,Desconegut,-1,Desconegut,Desconegut,Dimecres,2017,6,Juny,28,18,Canvi de carril sense precaució,Tarda,433440.23,4584188.34,2.202566,41.404640


In [20]:
df_accidents[df_accidents['Nom_carrer'].str.contains("Número 6 Zona Franca",regex=False)]

,Numero_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,NK_Any,Mes_any,Nom_mes,Dia_mes,Hora_dia,Descripcio_torn,Descripcio_causa_conductor,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud,Latitud
3,2017S008779,-1,Desconegut,-1,Desconegut,-1,Número 6 Zona Franca / A Zona Franca ...,NaN,Dilluns,2017,10,Octubre,23,22,Desobeir altres senyals,Nit,427519.99,4575229.36,2.190955,41.406769
733,2017S008207,3,Sants-Montjuïc,12,la Marina del Prat Vermell,370531,Número 6 Zona Franca,0074 0074,Dijous,2017,10,Octubre,5,6,No determinada,Matí,426702.23,4575207.50,2.123049,41.323166
2183,2017S009768,3,Sants-Montjuïc,12,la Marina del Prat Vermell,370531,Número 6 Zona Franca,0052 0052,Divendres,2017,11,Novembre,24,9,Desobeir altres senyals,Matí,427519.99,4575229.36,2.132817,41.323437
3468,2017S006313,3,Sants-Montjuïc,12,la Marina del Prat Vermell,370531,Número 6 Zona Franca,0097 0111,Dimecres,2017,7,Juliol,19,17,Desconegut,Tarda,425967.66,4575584.75,2.114227,41.326497
4839,2017S002233,3,Sants-Montjuïc,12,la Marina del Prat Vermell,370531,Número 6 Zona Franca,0087 0087,Dilluns,2017,3,Març,20,13,Manca d'atenció a la conducció,Matí,426266.43,4576025.75,2.117743,41.330496
7597,2017S010516,3,Sants-Montjuïc,12,la Marina del Prat Vermell,370531,Número 6 Zona Franca,0074 0074,Dimecres,2017,12,Desembre,20,8,Manca d'atenció a la conducció,Matí,426702.23,4575207.50,2.123049,41.323166


# df_lloguer -- análisis y limpieza

In [21]:
report_df(df_lloguer)

df_lloguer.head(2)

El tamaño del dataset es 584 filas y 8 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Any             584 non-null    int64  
 1   Trimestre       584 non-null    int64  
 2   Codi_Districte  584 non-null    int64  
 3   Nom_Districte   584 non-null    object 
 4   Codi_Barri      584 non-null    int64  
 5   Nom_Barri       584 non-null    object 
 6   Lloguer_mitja   584 non-null    object 
 7   Preu            546 non-null    float64
dtypes: float64(1), int64(4), object(3)
memory usage: 36.6+ KB
None

Tenemos un total de 38 nulos


,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
0,2017,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99
1,2017,1,1,Ciutat Vella,2,el Barri Gòtic,Lloguer mitjà mensual (Euros/mes),905.26


In [22]:
df_lloguer.isnull().sum()

Any                0
Trimestre          0
Codi_Districte     0
Nom_Districte      0
Codi_Barri         0
Nom_Barri          0
Lloguer_mitja      0
Preu              38
dtype: int64

In [23]:
df_lloguer[df_lloguer['Preu'].isnull()]

,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
11,2017,1,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà mensual (Euros/mes),NaN
41,2017,1,7,Horta-Guinardó,42,la Clota,Lloguer mitjà mensual (Euros/mes),NaN
46,2017,1,8,Nou Barris,47,Can Peguera,Lloguer mitjà mensual (Euros/mes),NaN
55,2017,1,8,Nou Barris,56,Vallbona,Lloguer mitjà mensual (Euros/mes),NaN
57,2017,1,9,Sant Andreu,58,Baró de Viver,Lloguer mitjà mensual (Euros/mes),NaN
84,2017,2,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà mensual (Euros/mes),NaN
114,2017,2,7,Horta-Guinardó,42,la Clota,Lloguer mitjà mensual (Euros/mes),NaN
126,2017,2,8,Nou Barris,54,Torre Baró,Lloguer mitjà mensual (Euros/mes),NaN
130,2017,2,9,Sant Andreu,58,Baró de Viver,Lloguer mitjà mensual (Euros/mes),NaN
157,2017,3,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà mensual (Euros/mes),NaN


In [24]:
df_lloguer[df_lloguer['Preu'].isnull()]['Nom_Barri'].unique()

array(['la Marina del Prat Vermell', 'la Clota', 'Can Peguera',
       'Vallbona', 'Baró de Viver', 'Torre Baró'], dtype=object)

In [25]:
df_lloguer[df_lloguer['Nom_Barri']=='la Marina del Prat Vermell']

,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
11,2017,1,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà mensual (Euros/mes),NaN
84,2017,2,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà mensual (Euros/mes),NaN
157,2017,3,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà mensual (Euros/mes),NaN
230,2017,4,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà mensual (Euros/mes),NaN
303,2017,1,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà per superfície (Euros/m2 mes),NaN
376,2017,2,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà per superfície (Euros/m2 mes),NaN
449,2017,3,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà per superfície (Euros/m2 mes),NaN
522,2017,4,3,Sants-Montjuïc,12,la Marina del Prat Vermell,Lloguer mitjà per superfície (Euros/m2 mes),NaN


In [26]:
df_lloguer[df_lloguer['Nom_Districte']=='Sants-Montjuïc']['Codi_Barri'].unique()

array([11, 12, 13, 14, 15, 16, 17, 18], dtype=int64)

In [27]:
df_lloguer.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
Any,584.0,2017.000000,0.000000,2017.00,2017.000,2017.000,2017.0000,2017.00
Trimestre,584.0,2.500000,1.118992,1.00,1.750,2.500,3.2500,4.00
Codi_Districte,584.0,6.246575,2.790726,1.00,4.000,7.000,8.0000,10.00
Codi_Barri,584.0,37.000000,21.089371,1.00,19.000,37.000,55.0000,73.00
Preu,546.0,421.009231,446.178147,4.45,12.645,161.565,770.3475,1882.42


In [28]:
df_lloguer.describe(exclude=np.number).T

,count,unique,top,freq
Nom_Districte,584,10,Nou Barris,104
Nom_Barri,584,73,el Raval,8
Lloguer_mitja,584,2,Lloguer mitjà mensual (Euros/mes),292


#### Observamos que hay columnas sin varianza como el caso del año,siempre es 2017, por lo tanto no nos aporta información útil de cara a hacer un futuro modelo y por eso lo eliminamos utilizando esa función lambda.

In [29]:
list_vars_non_variance = [column for column in df_lloguer.columns if df_lloguer[column].nunique(dropna = False) == 1]
df_lloguer.drop(list_vars_non_variance, axis = 1, inplace = True)

In [30]:
df_lloguer.head(1)

,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
0,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99


In [31]:
df_superficie.columns

Index(['Any', 'Codi_districte', 'Nom_districte', 'Codi_barri', 'Nom_barri',
       'Superficie_mitjana_habitatge_(m2)'],
      dtype='object')

In [32]:
df_lloguer.columns

Index(['Trimestre', 'Codi_Districte', 'Nom_Districte', 'Codi_Barri',
       'Nom_Barri', 'Lloguer_mitja', 'Preu'],
      dtype='object')

#### Hemos visto que el DataFrame de lloguer esta compuesto por 584 filas, 292 con precio medio mensual del alquiler y los otros 292 son exactamente las mismas filas varian que el precio esta puesto por m2. Así que vamos a generar un "id" único para cada fila, ya que cada fila corresponde a un trimestre, distrito y barrio distintos. 

#### Después colocaremos esas 292 filas con precio por m2 y lo convertiremos en una columna, de tal manera que nos quedaremos con un Dataframe de lloguer de 292 filas y con valores de precio medio mensual y precio por m2 en columnas diferenciadas

In [33]:
# Creación de id único con Trimestre, Distrito y Barrio
df_lloguer['id'] = df_lloguer['Trimestre'].apply(str) + "_" + df_lloguer['Codi_Districte'].apply(str)  + "_" + df_lloguer['Codi_Barri'].apply(str) 

In [34]:
df_lloguer.set_index('id',inplace=True)

In [35]:
# Filtramos por los valores de precio medio por superficie y nos quedamos con el precio, 
# le cambiamos el nombre a la columna para después merge con la otra parte del df y tenerlo bien identificado.
preu_superficie = df_lloguer[df_lloguer['Lloguer_mitja']=="Lloguer mitjà per superfície (Euros/m2 mes)"][['Preu']]
preu_superficie.rename(columns = {"Preu":"Preu_lloguer_mitja_superficie"},inplace=True)

In [36]:
preu_superficie #imprimimos para ver cómo queda

,Preu_lloguer_mitja_superficie
id,
1_1_1,13.50
1_1_2,14.14
1_1_3,19.46
1_1_4,15.01
1_2_5,12.77
...,...
4_10_69,16.51
4_10_70,10.40
4_10_71,12.66


In [37]:
# Filtramos por la otra mitad del dataset y cambiamos también el nombre de la columna para tenerlo bien identificado
df_lloguer_unique = df_lloguer[df_lloguer['Lloguer_mitja']=="Lloguer mitjà mensual (Euros/mes)"]
df_lloguer_unique.rename(columns = {"Preu":"Preu_lloguer_mitja_mensual"},inplace=True)

In [38]:
# Hacemos merge a través del 'id' que hemos creado antes e imprimimos el shape antes y 
# después para asegurar de no hacer un cartesiano
print(df_lloguer_unique.shape)

df_lloguer_unique = pd.merge(df_lloguer_unique,preu_superficie, on = 'id', how='left')

print(df_lloguer_unique.shape)

(292, 7)
(292, 8)


In [39]:
df_lloguer_unique.isnull().sum() #comprobamos los nulos y vemos que coinciden con los que teniamos desde un inicio

Trimestre                         0
Codi_Districte                    0
Nom_Districte                     0
Codi_Barri                        0
Nom_Barri                         0
Lloguer_mitja                     0
Preu_lloguer_mitja_mensual       19
Preu_lloguer_mitja_superficie    19
dtype: int64

#### Realizamos una comprobación visual para ver que coincide el cambio realizado

In [40]:
df_lloguer_unique[(df_lloguer_unique['Codi_Barri']==1)&(df_lloguer_unique['Codi_Districte']==1)]

,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu_lloguer_mitja_mensual,Preu_lloguer_mitja_superficie
id,,,,,,,,
1_1_1,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99,13.50
2_1_1,2,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),714.29,13.69
3_1_1,3,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),764.85,13.97
4_1_1,4,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),776.73,14.42


In [41]:
df_lloguer[(df_lloguer['Codi_Barri']==1)&(df_lloguer['Codi_Districte']==1)]

,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Lloguer_mitja,Preu
id,,,,,,,
1_1_1,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),734.99
2_1_1,2,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),714.29
3_1_1,3,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),764.85
4_1_1,4,1,Ciutat Vella,1,el Raval,Lloguer mitjà mensual (Euros/mes),776.73
1_1_1,1,1,Ciutat Vella,1,el Raval,Lloguer mitjà per superfície (Euros/m2 mes),13.50
2_1_1,2,1,Ciutat Vella,1,el Raval,Lloguer mitjà per superfície (Euros/m2 mes),13.69
3_1_1,3,1,Ciutat Vella,1,el Raval,Lloguer mitjà per superfície (Euros/m2 mes),13.97
4_1_1,4,1,Ciutat Vella,1,el Raval,Lloguer mitjà per superfície (Euros/m2 mes),14.42


#### Vamos a enriquecer nuestro dataset con los m2 promedio de las viviendas según el barrio, distrito de Barcelona

#### Para ello, hemos conseguido otro archivo que contiene esa infromación y dispone de columnas similares a nuestro Dataframe actual por lo que haremos unos cambios en los nombres y así poder hacer el merge

In [42]:
# Define una función para cambiar la primera letra después de "_"
def cambiar_letra_despues_de_guion_bajo(columna):
    return re.sub(r'_(.)', lambda x: '_' + x.group(1).upper(), columna)

In [43]:
# Aplica la función a todas las columnas
df_superficie.columns = df_superficie.columns.map(cambiar_letra_despues_de_guion_bajo)

In [44]:
df_superficie

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Superficie_Mitjana_Habitatge_(m2)
0,2017,1,Ciutat Vella,1,el Raval,65.6
1,2017,1,Ciutat Vella,2,el Barri Gòtic,88.2
2,2017,1,Ciutat Vella,3,la Barceloneta,46.7
3,2017,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",72.6
4,2017,2,Eixample,5,el Fort Pienc,81.9
...,...,...,...,...,...,...
68,2017,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,86.9
69,2017,10,Sant Martí,70,el Besòs i el Maresme,63.8
70,2017,10,Sant Martí,71,Provençals del Poblenou,75.1
71,2017,10,Sant Martí,72,Sant Martí de Provençals,75.1


In [45]:
df_lloguer_unique.reset_index(inplace=True) #Reseteamos indice para no perderlo al hacer el merge

In [46]:
print(df_lloguer_unique.shape) #Para comprobar que no hace un cartesiano medimos el tamaño antes y después

df_lloguer_unique = pd.merge(
    df_lloguer_unique,
    df_superficie,
    how='left',
    on = ["Codi_Districte","Nom_Districte","Codi_Barri","Nom_Barri"],
    )

print(df_lloguer_unique.shape) #Para comprobar que no hace un cartesiano medimos el tamaño antes y después

(292, 9)
(292, 11)


In [47]:
df_lloguer_unique.set_index('id',inplace=True) #Volvemos a poner el id en el índice

In [48]:
df_lloguer_unique.drop(["Any","Lloguer_mitja"],axis=1,inplace=True) #Eliminamos columnas que no son necesarias

In [49]:
df_lloguer_unique

,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Preu_lloguer_mitja_mensual,Preu_lloguer_mitja_superficie,Superficie_Mitjana_Habitatge_(m2)
id,,,,,,,,
1_1_1,1,1,Ciutat Vella,1,el Raval,734.99,13.50,65.6
1_1_2,1,1,Ciutat Vella,2,el Barri Gòtic,905.26,14.14,88.2
1_1_3,1,1,Ciutat Vella,3,la Barceloneta,722.78,19.46,46.7
1_1_4,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",895.28,15.01,72.6
1_2_5,1,2,Eixample,5,el Fort Pienc,871.08,12.77,81.9
...,...,...,...,...,...,...,...,...
4_10_69,4,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,1233.01,16.51,86.9
4_10_70,4,10,Sant Martí,70,el Besòs i el Maresme,621.32,10.40,63.8
4_10_71,4,10,Sant Martí,71,Provençals del Poblenou,864.09,12.66,75.1


# Imputación de valores nulos en precio

In [50]:
df_lloguer_unique.isnull().sum() #Seguimos teniendo la misma cantidad a pesar de los cambios, así que es buena señal

Trimestre                             0
Codi_Districte                        0
Nom_Districte                         0
Codi_Barri                            0
Nom_Barri                             0
Preu_lloguer_mitja_mensual           19
Preu_lloguer_mitja_superficie        19
Superficie_Mitjana_Habitatge_(m2)     0
dtype: int64

#### Usaremos el KNNImputer para imputar los precios en función de sus vecinos más cercanos, de esta manera podemos imputar nulos y el agoritmo buscará aquellos más parecidos (barrio, distrito,etc), para imputar valores 

In [51]:
imputer = KNNImputer()

In [52]:
df_lloguer_unique.describe(include = np.number).columns

Index(['Trimestre', 'Codi_Districte', 'Codi_Barri',
       'Preu_lloguer_mitja_mensual', 'Preu_lloguer_mitja_superficie',
       'Superficie_Mitjana_Habitatge_(m2)'],
      dtype='object')

In [53]:
df_lloguer_unique_num = df_lloguer_unique[
    ['Trimestre', 
     'Codi_Districte', 
     'Codi_Barri',
    'Preu_lloguer_mitja_mensual', 
    'Preu_lloguer_mitja_superficie',
    'Superficie_Mitjana_Habitatge_(m2)']]

In [54]:
df_lloguer_unique_num = imputer.fit_transform(df_lloguer_unique_num)

In [55]:
df_lloguer_unique_num.isnull().sum() #Ya hemos imputado los valores nulos

Trimestre                            0
Codi_Districte                       0
Codi_Barri                           0
Preu_lloguer_mitja_mensual           0
Preu_lloguer_mitja_superficie        0
Superficie_Mitjana_Habitatge_(m2)    0
dtype: int64

#### Vamos a concatenar las otras columnas que no hemos seleccionado antes para poder concatenar con el df_poblacio

#### Como no hemos alterado el orden del dataset podemos hacer un concat en axis=1, podemos estar seguros de que seguirá el mismo orden que debería

In [56]:
df_lloguer_unique_clean = pd.concat([df_lloguer_unique_num,df_lloguer_unique[['Nom_Districte','Nom_Barri']]],axis=1)

In [57]:
df_lloguer_unique_clean

,Trimestre,Codi_Districte,Codi_Barri,Preu_lloguer_mitja_mensual,Preu_lloguer_mitja_superficie,Superficie_Mitjana_Habitatge_(m2),Nom_Districte,Nom_Barri
id,,,,,,,,
1_1_1,1.0,1.0,1.0,734.99,13.50,65.6,Ciutat Vella,el Raval
1_1_2,1.0,1.0,2.0,905.26,14.14,88.2,Ciutat Vella,el Barri Gòtic
1_1_3,1.0,1.0,3.0,722.78,19.46,46.7,Ciutat Vella,la Barceloneta
1_1_4,1.0,1.0,4.0,895.28,15.01,72.6,Ciutat Vella,"Sant Pere, Santa Caterina i la Ribera"
1_2_5,1.0,2.0,5.0,871.08,12.77,81.9,Eixample,el Fort Pienc
...,...,...,...,...,...,...,...,...
4_10_69,4.0,10.0,69.0,1233.01,16.51,86.9,Sant Martí,Diagonal Mar i el Front Marítim del Poblenou
4_10_70,4.0,10.0,70.0,621.32,10.40,63.8,Sant Martí,el Besòs i el Maresme
4_10_71,4.0,10.0,71.0,864.09,12.66,75.1,Sant Martí,Provençals del Poblenou


# df_poblacio -- análisis y limpieza

In [58]:
report_df(df_poblacio)

df_poblacio.head(2)

El tamaño del dataset es 18980 filas y 7 columnas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18980 entries, 0 to 18979
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codi_Districte  18980 non-null  int64 
 1   Nom_Districte   18980 non-null  object
 2   Codi_Barri      18980 non-null  int64 
 3   Nom_Barri       18980 non-null  object
 4   Concepte        18980 non-null  object
 5   Rang_soroll     18980 non-null  object
 6   Valor           18980 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.0+ MB
None

Tenemos un total de 0 nulos


,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Concepte,Rang_soroll,Valor
0,1,Ciutat Vella,1,el Raval,TOTAL_D,<40 dB,7.73%
1,1,Ciutat Vella,1,el Raval,TOTAL_D,40-45 dB,26.98%


In [59]:
df_poblacio.describe(include = np.number).T

,count,mean,std,min,25%,50%,75%,max
Codi_Districte,18980.0,6.246575,2.788409,1.0,4.0,7.0,8.0,10.0
Codi_Barri,18980.0,37.000000,21.071863,1.0,19.0,37.0,55.0,73.0


In [60]:
df_poblacio.describe(exclude=np.number).T

,count,unique,top,freq
Nom_Districte,18980,10,Nou Barris,3380
Nom_Barri,18980,73,el Raval,260
Concepte,18980,26,TOTAL_D,730
Rang_soroll,18980,10,<40 dB,1898
Valor,18980,3104,0.00%,9953


In [61]:
df_poblacio.isnull().sum()

Codi_Districte    0
Nom_Districte     0
Codi_Barri        0
Nom_Barri         0
Concepte          0
Rang_soroll       0
Valor             0
dtype: int64

In [62]:
df_poblacio[(df_poblacio['Codi_Districte']==1) & (df_poblacio['Codi_Barri']==1)]

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Concepte,Rang_soroll,Valor
0,1,Ciutat Vella,1,el Raval,TOTAL_D,<40 dB,7.73%
1,1,Ciutat Vella,1,el Raval,TOTAL_D,40-45 dB,26.98%
2,1,Ciutat Vella,1,el Raval,TOTAL_D,45-50 dB,7.38%
3,1,Ciutat Vella,1,el Raval,TOTAL_D,50-55 dB,11.97%
4,1,Ciutat Vella,1,el Raval,TOTAL_D,55-60 dB,19.85%
...,...,...,...,...,...,...,...
18255,1,Ciutat Vella,1,el Raval,PARCS_D,60-65 dB,0.00%
18256,1,Ciutat Vella,1,el Raval,PARCS_D,65-70 dB,0.00%
18257,1,Ciutat Vella,1,el Raval,PARCS_D,70-75 dB,0.00%
18258,1,Ciutat Vella,1,el Raval,PARCS_D,75-80 dB,0.00%


In [63]:
df_poblacio[(df_poblacio['Codi_Districte']==1)]['Codi_Barri'].nunique()

4

In [64]:
df_poblacio['Codi_Districte'].nunique()

10

In [65]:
df_poblacio['Codi_Barri'].nunique()

73

In [66]:
df_poblacio['Rang_soroll'].nunique()

10

In [105]:
df_poblacio['Concepte'].unique() 
#Juntaremos etiquetas y haremos menos grupos en Conceptos, ya que muchos forman parte de grupos similares

array(['TOTAL_D', 'TOTAL_E', 'TOTAL_N', 'TOTAL_DEN', 'TRANSIT_D',
       'TRANSIT_E', 'TRANSIT_N', 'TRANSIT_DEN', 'GI_TR_D', 'GI_TR_E',
       'GI_TR_N', 'GI_TR_DEN', 'FFCC_D', 'FFCC_E', 'FFCC_N', 'FFCC_DEN',
       'INDUST_D', 'INDUST_E', 'INDUST_N', 'INDUST_DEN', 'VIANANTS_D',
       'VIANANTS_E', 'OCI_N', 'PATIS_D', 'PATIS_E', 'PARCS_D'],
      dtype=object)

In [69]:
# Clasifficación general de Concepte, según la web de www.barcelona.cat, lo divide en estas 9 categorias diferentes:

# TOTAL
# PATIS
# OCI
# VIANANTS
# INDUST
# FFCC
# GI
# TRANSIT
# PARCS

In [68]:
df_poblacio.index.nunique()

18980

In [70]:
df_poblacio.columns

Index(['Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri',
       'Concepte', 'Rang_soroll', 'Valor'],
      dtype='object')

In [108]:
# agrupación de etiquetas

juntar_etiquetas(df_poblacio, 'Concepte', 'TOTAL', 'TOTAL')
juntar_etiquetas(df_poblacio, 'Concepte', 'TRANSIT', 'TRANSIT')
juntar_etiquetas(df_poblacio, 'Concepte', 'GI_TR', 'GI_TR')
juntar_etiquetas(df_poblacio, 'Concepte', 'FFCC', 'FFCC')
juntar_etiquetas(df_poblacio, 'Concepte', 'INDUST', 'INDUST')
juntar_etiquetas(df_poblacio, 'Concepte', 'VIANANTS', 'VIANANTS')
juntar_etiquetas(df_poblacio, 'Concepte', 'OCI', 'OCI')
juntar_etiquetas(df_poblacio, 'Concepte', 'PATIS', 'PATIS')
juntar_etiquetas(df_poblacio, 'Concepte', 'PARCS', 'PARCS')

In [110]:
df_poblacio['Concepte'].value_counts()

Concepte
TOTAL       2920
TRANSIT     2920
GI_TR       2920
FFCC        2920
INDUST      2920
VIANANTS    1460
PATIS       1460
OCI          730
PARCS        730
Name: count, dtype: int64

In [111]:
df_poblacio['Valor'] = df_poblacio['Valor'].str.split("%",expand=True)[[0]]
df_poblacio.rename(columns = {'Valor':'Valor_porcentaje'},inplace=True)

In [113]:
df_poblacio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18980 entries, 0 to 18979
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Codi_Districte    18980 non-null  int64 
 1   Nom_Districte     18980 non-null  object
 2   Codi_Barri        18980 non-null  int64 
 3   Nom_Barri         18980 non-null  object
 4   Concepte          18980 non-null  object
 5   Rang_soroll       18980 non-null  object
 6   Valor_porcentaje  18980 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.0+ MB


In [114]:
df_poblacio['Valor_porcentaje'] = df_poblacio['Valor_porcentaje'].astype(float) #Convertimos a numerico los valores

In [116]:
#Creación de id único con Trimestre, Distrito y Barrio
df_poblacio['id'] = df_poblacio['Codi_Districte'].apply(str)  + "_" + df_poblacio['Codi_Barri'].apply(str) + "_" + df_poblacio['Rang_soroll']

df_poblacio.set_index('id',inplace=True)

In [117]:
df_poblacio['media_valor_concepte'] = df_poblacio.groupby([df_poblacio.index,'Concepte'])[['Valor_porcentaje']].transform(np.mean)

In [118]:
df_poblacio[df_poblacio['Concepte'] == 'VIANANTS']

,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Concepte,Rang_soroll,Valor_porcentaje,media_valor_concepte
id,,,,,,,,
1_1_<40 dB,1,Ciutat Vella,1,el Raval,VIANANTS,<40 dB,57.69,58.410
1_1_40-45 dB,1,Ciutat Vella,1,el Raval,VIANANTS,40-45 dB,13.35,15.075
1_1_45-50 dB,1,Ciutat Vella,1,el Raval,VIANANTS,45-50 dB,10.90,9.970
1_1_50-55 dB,1,Ciutat Vella,1,el Raval,VIANANTS,50-55 dB,10.81,9.420
1_1_55-60 dB,1,Ciutat Vella,1,el Raval,VIANANTS,55-60 dB,6.47,6.245
...,...,...,...,...,...,...,...,...
10_73_60-65 dB,10,Sant Martí,73,la Verneda i la Pau,VIANANTS,60-65 dB,0.00,0.000
10_73_65-70 dB,10,Sant Martí,73,la Verneda i la Pau,VIANANTS,65-70 dB,0.00,0.000
10_73_70-75 dB,10,Sant Martí,73,la Verneda i la Pau,VIANANTS,70-75 dB,0.00,0.000


In [ ]:
df_poblacio.drop('Valor_porcentaje',axis=1,inplace=True)
df_poblacio.reset_index(drop=True,inplace=True)
df_poblacio

In [ ]:
df_poblacio = pd.pivot_table(df_poblacio,index = ['Codi_Districte', 'Nom_Districte', 'Codi_Barri', 'Nom_Barri','Rang_soroll'], columns = 'Concepte',values = "media_valor_concepte")
df_poblacio.reset_index(inplace=True)

In [125]:
df_poblacio[(df_poblacio['Codi_Districte']==1) & (df_poblacio['Codi_Barri']==1)] 
# Comprobamos que dentro de las columnas de concepto nos de el 100% a los largo del rango de ruido

Concepte,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Rang_soroll,FFCC,GI_TR,INDUST,OCI,PARCS,PATIS,TOTAL,TRANSIT,VIANANTS
0,1,Ciutat Vella,1,el Raval,40-45 dB,0.0,5.3400,0.0,4.97,0.0,0.000,20.2875,23.1625,15.075
1,1,Ciutat Vella,1,el Raval,45-50 dB,0.0,1.7425,0.0,2.53,0.0,0.000,8.9775,9.3350,9.970
2,1,Ciutat Vella,1,el Raval,50-55 dB,0.0,0.8500,0.0,1.80,0.0,0.825,11.8725,12.8425,9.420
3,1,Ciutat Vella,1,el Raval,55-60 dB,0.0,0.6025,0.0,5.79,0.0,1.335,16.3775,15.2550,6.245
4,1,Ciutat Vella,1,el Raval,60-65 dB,0.0,0.7325,0.0,8.88,0.0,0.890,14.8850,10.7925,0.880
5,1,Ciutat Vella,1,el Raval,65-70 dB,0.0,0.6500,0.0,0.14,0.0,0.000,7.4325,5.3225,0.000
6,1,Ciutat Vella,1,el Raval,70-75 dB,0.0,0.6750,0.0,0.00,0.0,0.000,2.0925,1.7300,0.000
7,1,Ciutat Vella,1,el Raval,75-80 dB,0.0,0.0000,0.0,0.00,0.0,0.000,0.0250,0.0250,0.000
8,1,Ciutat Vella,1,el Raval,<40 dB,100.0,89.4075,100.0,75.89,100.0,96.950,18.0500,21.5350,58.410
9,1,Ciutat Vella,1,el Raval,>=80 dB,0.0,0.0000,0.0,0.00,0.0,0.000,0.0000,0.0000,0.000


In [130]:
# Realizar Label Encoder a Rang Soroll

le = LabelEncoder()
df_poblacio['Rang_Soroll']= le.fit_transform(df_poblacio['Rang_Soroll'])

KeyError: 'Rang_Soroll'